# World Cup 2022 winner classifier
Imagine being able to predict which team will win during a match. All your bets would win and you'd be rich! Unfortunately, football, like any sport, is unpredictable, especially at the World Cup.


An attempt to build such an ML model is doomed to failure, the only question is how much? The project's task is to predict the winner of a given match, and thanks to the ongoing championships, I can check the accuracy of the model.

Datasets used: 
FIFA ranking -  https://www.kaggle.com/code/agostontorok/soccer-world-cup-2018-winner/data   
football results -  https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None

In [2]:
#importing data
fifa_ranking = pd.read_csv(r"data\world cup 2022\fifa_ranking-2022-10-06.csv")
matches =  pd.read_csv(r"data\world cup 2022\results.csv")

## Matches data 

In [13]:
matches

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44336,2022-12-04,England,Senegal,3,0,FIFA World Cup,Al Khor,Qatar,True
44337,2022-12-05,Japan,Croatia,1,1,FIFA World Cup,Al Wakrah,Qatar,True
44338,2022-12-05,Brazil,South Korea,4,1,FIFA World Cup,Doha,Qatar,True
44339,2022-12-06,Morocco,Spain,0,0,FIFA World Cup,Al Rayyan,Qatar,True


In [22]:
#renaming columns 
matches.rename(columns = {'home_team':'team_1', 'away_team':'team_2','home_score':'team_1_score', 'away_score':'team_2_score'}, inplace = True)

In [23]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44341 entries, 0 to 44340
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          44341 non-null  datetime64[ns]
 1   team_1        44341 non-null  object        
 2   team_2        44341 non-null  object        
 3   team_1_score  44341 non-null  int64         
 4   team_2_score  44341 non-null  int64         
 5   tournament    44341 non-null  object        
 6   city          44341 non-null  object        
 7   country       44341 non-null  object        
 8   neutral       44341 non-null  bool          
 9   result        44341 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(3), object(5)
memory usage: 3.1+ MB


In [24]:
# date dtype is object, let's convert it to datetime
matches["date"] = pd.to_datetime(matches["date"], format = '%Y-%m-%d')

In [25]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1
...,...,...,...,...,...,...,...,...,...,...
44336,2022-12-04,England,Senegal,3,0,FIFA World Cup,Al Khor,Qatar,True,1
44337,2022-12-05,Japan,Croatia,1,1,FIFA World Cup,Al Wakrah,Qatar,True,1
44338,2022-12-05,Brazil,South Korea,4,1,FIFA World Cup,Doha,Qatar,True,1
44339,2022-12-06,Morocco,Spain,0,0,FIFA World Cup,Al Rayyan,Qatar,True,1


## Ranking data

In [26]:
fifa_ranking

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,date
0,1,Germany,GER,57.00,0.00,0,UEFA,1992-12-31
1,96,Syria,SYR,11.00,0.00,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.00,0.00,0,CAF,1992-12-31
3,99,Latvia,LVA,10.00,0.00,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.00,0.00,0,CAF,1992-12-31
...,...,...,...,...,...,...,...,...
63911,74,El Salvador,SLV,1330.51,1333.48,3,CONCACAF,2022-10-06
63912,75,Oman,OMA,1320.29,1323.03,0,AFC,2022-10-06
63913,76,Israel,ISR,1316.55,1316.35,0,UEFA,2022-10-06
63914,78,Georgia,GEO,1307.34,1296.46,-4,UEFA,2022-10-06


In [27]:
#renaming columns 
fifa_ranking.rename(columns = {'rank_date':'date'}, inplace = True)

In [28]:
fifa_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rank             63916 non-null  int64         
 1   country_full     63916 non-null  object        
 2   country_abrv     63916 non-null  object        
 3   total_points     63916 non-null  float64       
 4   previous_points  63916 non-null  float64       
 5   rank_change      63916 non-null  int64         
 6   confederation    63916 non-null  object        
 7   date             63916 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 3.9+ MB


In [29]:
# date dtype is object, let's convert it to datetime
fifa_ranking["date"] = pd.to_datetime(fifa_ranking["date"], format = '%Y-%m-%d')

## Final data set

In [30]:
#making target variable
matches["result"] = 1
matches.loc[matches.team_1_score > matches.team_2_score, "result" ] = 0
matches.loc[matches.team_1_score < matches.team_2_score, "result" ] = 2

In [31]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,0
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,0
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,0
...,...,...,...,...,...,...,...,...,...,...
44336,2022-12-04,England,Senegal,3,0,FIFA World Cup,Al Khor,Qatar,True,0
44337,2022-12-05,Japan,Croatia,1,1,FIFA World Cup,Al Wakrah,Qatar,True,1
44338,2022-12-05,Brazil,South Korea,4,1,FIFA World Cup,Doha,Qatar,True,0
44339,2022-12-06,Morocco,Spain,0,0,FIFA World Cup,Al Rayyan,Qatar,True,1


In [32]:
#the world cup is on a neutral ground, so we should remove the other options
# However, it is worth keeping the data. We will see later if this was good choice.

In [33]:
#the first ranking date appeared in 1993, because of that we have to drop matches before this date
fifa_ranking.sort_values("date")

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,date
0,1,Germany,GER,57.00,0.00,0,UEFA,1992-12-31
95,74,Madagascar,MAD,18.00,0.00,0,CAF,1992-12-31
96,2,Italy,ITA,57.00,0.00,0,UEFA,1992-12-31
97,3,Brazil,BRA,56.00,0.00,0,CONMEBOL,1992-12-31
98,4,Sweden,SWE,56.00,0.00,0,UEFA,1992-12-31
...,...,...,...,...,...,...,...,...
63778,209,British Virgin Islands,VGB,809.32,809.32,0,CONCACAF,2022-10-06
63779,107,Kosovo,KOS,1187.01,1183.90,1,UEFA,2022-10-06
63780,200,Eritrea,ERI,855.56,855.56,-1,CAF,2022-10-06
63782,186,São Tomé and Príncipe,STP,908.18,908.18,-1,CAF,2022-10-06


In [34]:
first_date = fifa_ranking.sort_values("date").iloc[0].date
matches = matches[matches.date >= first_date]
first_date

Timestamp('1992-12-31 00:00:00')

In [35]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result
17712,1993-01-01,Ghana,Mali,1,1,Friendly,Libreville,Gabon,True,1
17713,1993-01-02,Gabon,Burkina Faso,1,1,Friendly,Libreville,Gabon,False,1
17714,1993-01-02,Kuwait,Lebanon,2,0,Friendly,Kuwait City,Kuwait,False,0
17715,1993-01-03,Burkina Faso,Mali,1,0,Friendly,Libreville,Gabon,True,0
17716,1993-01-03,Gabon,Ghana,2,3,Friendly,Libreville,Gabon,False,2
...,...,...,...,...,...,...,...,...,...,...
44336,2022-12-04,England,Senegal,3,0,FIFA World Cup,Al Khor,Qatar,True,0
44337,2022-12-05,Japan,Croatia,1,1,FIFA World Cup,Al Wakrah,Qatar,True,1
44338,2022-12-05,Brazil,South Korea,4,1,FIFA World Cup,Doha,Qatar,True,0
44339,2022-12-06,Morocco,Spain,0,0,FIFA World Cup,Al Rayyan,Qatar,True,1


In [36]:
#writing a function to assign the appropriate index from the fifa_ranking df, where the team's rank is located
def add_ranking_index(team1, team2, date):
    ranking = fifa_ranking[fifa_ranking.date <= date]
    rank_team1 = ranking[ranking.country_full == team1]
    rank_team2 = ranking[ranking.country_full == team2]
    
    if(rank_team1.empty):
        team1_index = np.NaN
    else:
        team1_index = rank_team1.sort_values("date", ascending=False).index[0]
        
    if(rank_team2.empty):
        team2_index = np.NaN
    else:
        team2_index = rank_team2.sort_values("date", ascending=False).index[0]
        
    return pd.Series([team1_index, team2_index])

In [37]:
#it seems to be working
add_ranking_index(matches.iloc[-1].team_1, matches.iloc[-1].team_2, matches.iloc[-1].date)

0    63845
1    63851
dtype: int64

In [38]:
#scale it up 
matches[['id1', 'id2']] = matches.apply(lambda row: add_ranking_index(row["team_1"], row["team_2"], row["date"]), axis=1)

In [39]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result,id1,id2
17712,1993-01-01,Ghana,Mali,1,1,Friendly,Libreville,Gabon,True,1,132.0,125.0
17713,1993-01-02,Gabon,Burkina Faso,1,1,Friendly,Libreville,Gabon,False,1,130.0,2.0
17714,1993-01-02,Kuwait,Lebanon,2,0,Friendly,Kuwait City,Kuwait,False,0,127.0,NaN
17715,1993-01-03,Burkina Faso,Mali,1,0,Friendly,Libreville,Gabon,True,0,2.0,125.0
17716,1993-01-03,Gabon,Ghana,2,3,Friendly,Libreville,Gabon,False,2,130.0,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...
44336,2022-12-04,England,Senegal,3,0,FIFA World Cup,Al Khor,Qatar,True,0,63841.0,63854.0
44337,2022-12-05,Japan,Croatia,1,1,FIFA World Cup,Al Wakrah,Qatar,True,1,63836.0,63848.0
44338,2022-12-05,Brazil,South Korea,4,1,FIFA World Cup,Doha,Qatar,True,0,63809.0,NaN
44339,2022-12-06,Morocco,Spain,0,0,FIFA World Cup,Al Rayyan,Qatar,True,1,63858.0,63843.0


In [40]:
#some of the values are NaN, let's see why
matches.loc[matches.isnull().values, ["date", "team_1", "team_2", "id1", "id2" ]]
#looks like first nan values are due to the lack of country in ranking (future rankings include more countries)

,date,team_1,team_2,id1,id2
17714,1993-01-02,Kuwait,Lebanon,127.0,NaN
17721,1993-01-10,DR Congo,Cameroon,NaN,77.0
17728,1993-01-17,Botswana,Ivory Coast,41.0,NaN
17732,1993-01-17,Namibia,Madagascar,NaN,95.0
17736,1993-01-19,North Korea,Bolivia,NaN,29.0
...,...,...,...,...,...
44320,2022-11-29,Iran,United States,NaN,NaN
44320,2022-11-29,Iran,United States,NaN,NaN
44332,2022-12-02,South Korea,Portugal,NaN,63845.0
44333,2022-12-03,Netherlands,United States,63844.0,NaN


In [42]:
#but what about other nan values? Maybe problem is with country name?
matches.loc[matches.id1.isnull().values, "team_1"].value_counts()[:10]
# matches.loc[matches.id2.isnull().values, "team_2"].value_counts()[:10]

United States            340
South Korea              271
Iran                     203
Ivory Coast              162
DR Congo                 127
Saint Kitts and Nevis     95
Martinique                91
North Korea               80
North Macedonia           73
Guadeloupe                71
Name: team_1, dtype: int64

In [43]:
np.sort(fifa_ranking.country_full.unique())

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Cape Verde Islands', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China PR',
       'Chinese Taipei', 'Colombia', 'Comoros', 'Congo', 'Congo DR',
       'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curacao',
       'Curaçao', 'Cyprus', 'Czech Republic', 'Czechoslovakia',
       "Côte d'Ivoire", 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'England',
       'Equato

In [44]:
#United States is USA in FIFA
#South Korea is Korea Republic
#Iran is IR Iran
#Ivory Coast is Côte d'Ivoire
#North Macedonia is  FYR Macedonia
#DR Congo  is Congo or Congo DR
#North Korea is Korea DPR
#Northern Cyprus is Cyprus
#Eswatini is Swaziland
#Ireland is Republic of Ireland
#rest are insignificant teams and I will drop them

In [45]:
#replacing all wrong names to FIFA names 
matches.team_1.replace(['United States'], ['USA'], inplace = True)
matches.team_2.replace(['United States'], ['USA'], inplace = True)

matches.team_1.replace(['South Korea'], ['Korea Republic'], inplace = True)
matches.team_2.replace(['South Korea'], ['Korea Republic'], inplace = True)

matches.team_1.replace(['Iran'], ['IR Iran'], inplace = True)
matches.team_2.replace(['Iran'], ['IR Iran'], inplace = True)

matches.team_1.replace(['Ivory Coast'], ["Côte d'Ivoire"], inplace = True)
matches.team_2.replace(['Ivory Coast'], ["Côte d'Ivoire"], inplace = True)

matches.team_1.replace(['North Macedonia'], ['FYR Macedonia'], inplace = True)
matches.team_2.replace(['North Macedonia'], ['FYR Macedonia'], inplace = True)

matches.team_1.replace(['DR Congo'], ['Congo'], inplace = True)
matches.team_2.replace(['DR Congo'], ['Congo'], inplace = True)

fifa_ranking.country_full.replace(['Congo DR'], ['Congo'], inplace = True)

matches.team_1.replace(['North Korea'], ['Korea DPR'], inplace = True)
matches.team_2.replace(['North Korea'], ['Korea DPR'], inplace = True)

matches.team_1.replace(['Northern Cyprus'], ['Cyprus'], inplace = True)
matches.team_2.replace(['Northern Cyprus'], ['Cyprus'], inplace = True)

matches.team_1.replace(['Eswatini'], ['Swaziland'], inplace = True)
matches.team_2.replace(['Eswatini'], ['Swaziland'], inplace = True)

matches.team_1.replace(['Ireland'], ['Republic of Ireland'], inplace = True)
matches.team_2.replace(['Ireland'], ['Republic of Ireland'], inplace = True)




In [47]:
#now make it with correct names
matches[["id1", "id2"]] = matches.apply(lambda row: add_ranking_index(row["team_1"], row["team_2"], row["date"]), axis=1)

In [48]:
matches.loc[matches.id1.isnull().values, "team_1"].value_counts()[:]

Saint Kitts and Nevis        95
Martinique                   91
Guadeloupe                   71
Cape Verde                   68
Saint Lucia                  63
                             ..
Madrid                        1
Sark                          1
Saint Pierre and Miquelon     1
Silesia                       1
Aymara                        1
Name: team_1, Length: 140, dtype: int64

In [49]:
#muuuuch better :)
matches.loc[matches.id2.isnull().values, "team_2"].value_counts()[:]

Kyrgyzstan       88
Martinique       87
Saint Lucia      84
Serbia           80
Cape Verde       78
                 ..
Andalusia         1
Liechtenstein     1
Armenia           1
Brittany          1
Aymara            1
Name: team_2, Length: 142, dtype: int64

In [102]:
#saving matches to avoid recalculation
# matches.to_csv(r".\data\world cup 2022\matches_with_rank.csv", index=False)
matches_rank = pd.read_csv(r".\data\world cup 2022\matches_with_rank.csv")

In [103]:
#droping all rows where is NaN id
matches_rank.dropna(axis=0, inplace=True)

In [104]:
#know it is time to merge this datasets on id
matches_merged = matches_rank.merge(fifa_ranking, how = 'inner', left_on = 'id1', right_index = True, suffixes = ['', '_1'])
matches_merged = matches_merged.merge(fifa_ranking, how = 'inner', left_on = 'id2', right_index = True, suffixes = ['', '_2'])

In [105]:
matches_merged

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result,...,confederation,date_1,rank_2,country_full_2,country_abrv_2,total_points_2,previous_points_2,rank_change_2,confederation_2,date_2
0,1993-01-01,Ghana,Mali,1,1,Friendly,Libreville,Gabon,True,1,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
477,1993-07-04,Gabon,Mali,0,2,Friendly,Moanda,Gabon,False,2,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
3,1993-01-03,Burkina Faso,Mali,1,0,Friendly,Libreville,Gabon,True,0,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
171,1993-04-09,Egypt,Mali,2,1,African Cup of Nations qualification,Cairo,Egypt,False,0,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
246,1993-04-25,Morocco,Mali,1,0,African Cup of Nations qualification,Casablanca,Morocco,False,0,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26616,2022-12-01,Canada,Morocco,1,2,FIFA World Cup,Doha,Qatar,True,2,...,CONCACAF,2022-10-06,22,Morocco,MAR,1563.50,1558.35,-1,CAF,2022-10-06
26626,2022-12-05,Brazil,Korea Republic,4,1,FIFA World Cup,Doha,Qatar,True,0,...,CONMEBOL,2022-10-06,28,Korea Republic,KOR,1530.30,1526.02,0,AFC,2022-10-06
26587,2022-11-24,Uruguay,Korea Republic,0,0,FIFA World Cup,Al Rayyan,Qatar,True,1,...,CONMEBOL,2022-10-06,28,Korea Republic,KOR,1530.30,1526.02,0,AFC,2022-10-06
26607,2022-11-29,Wales,England,0,3,FIFA World Cup,Al Rayyan,Qatar,True,2,...,UEFA,2022-10-06,5,England,ENG,1728.47,1737.46,0,UEFA,2022-10-06


In [106]:
#changing names for convenience
matches_merged.columns = ['date', 'team_1', 'team_2', 'team_1_score', 'team_2_score',
       'tournament', 'city', 'country', 'neutral', 'result', 'id1', 'id2',
       'rank_1', 'country_full_1', 'country_abrv_1', 'total_points_1',
       'previous_points_1', 'rank_change_1', 'confederation_1', 'date_1', 'rank_2',
       'country_full_2', 'country_abrv_2', 'total_points_2',
       'previous_points_2', 'rank_change_2', 'confederation_2', 'date_2']

In [107]:
#droping useless columns
matches_merged.drop(['team_1', 'team_2', 'team_1_score', 'team_2_score', 'city',
              'country', 'id1', 'id2', 'country_full_1',
              'country_abrv_1', 'date_1', 'country_full_2',
              'country_abrv_2', 'date_2'], axis = 1, inplace = True)

In [108]:
matches_merged

,date,tournament,neutral,result,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2
0,1993-01-01,Friendly,True,1,39,34.00,0.00,0,CAF,69,22.00,0.00,0,CAF
477,1993-07-04,Friendly,False,2,55,27.00,0.00,0,CAF,69,22.00,0.00,0,CAF
3,1993-01-03,Friendly,True,0,97,11.00,0.00,0,CAF,69,22.00,0.00,0,CAF
171,1993-04-09,African Cup of Nations qualification,False,0,21,45.00,0.00,0,CAF,69,22.00,0.00,0,CAF
246,1993-04-25,African Cup of Nations qualification,False,0,41,32.00,0.00,0,CAF,69,22.00,0.00,0,CAF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26616,2022-12-01,FIFA World Cup,True,2,41,1475.00,1473.82,-2,CONCACAF,22,1563.50,1558.35,-1,CAF
26626,2022-12-05,FIFA World Cup,True,0,1,1841.30,1837.56,0,CONMEBOL,28,1530.30,1526.02,0,AFC
26587,2022-11-24,FIFA World Cup,True,1,14,1638.71,1640.95,1,CONMEBOL,28,1530.30,1526.02,0,AFC
26607,2022-11-29,FIFA World Cup,True,2,19,1569.82,1582.13,0,UEFA,5,1728.47,1737.46,0,UEFA


## Preprocessing

In [109]:
#some feature engineering

#time delta since the last date in the set
matches_merged["date"] = pd.to_datetime(matches_merged["date"], format = '%Y-%m-%d')
matches_merged["time_delta"] = (matches_merged["date"].values - np.datetime64('2022-11-20')).astype('timedelta64[M]')
matches_merged["time_delta"] = matches_merged["time_delta"].apply(lambda x: x / np.timedelta64(1,'M'))

#year
matches_merged['year'] = matches_merged["date"].dt.year
#month
matches_merged['month'] = matches_merged["date"].dt.month
#day of week
matches_merged['day_of_week'] = matches_merged["date"].dt.dayofweek
#day of year
matches_merged['day_of_year'] = matches_merged["date"].dt.dayofyear

In [110]:
matches_merged

,date,tournament,neutral,result,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2,time_delta,year,month,day_of_week,day_of_year
0,1993-01-01,Friendly,True,1,39,34.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318,1993,1,4,1
477,1993-07-04,Friendly,False,2,55,27.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-353.025729,1993,7,6,185
3,1993-01-03,Friendly,True,0,97,11.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318,1993,1,6,3
171,1993-04-09,African Cup of Nations qualification,False,0,21,45.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-356.048379,1993,4,4,99
246,1993-04-25,African Cup of Nations qualification,False,0,41,32.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-355.029877,1993,4,6,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26616,2022-12-01,FIFA World Cup,True,2,41,1475.00,1473.82,-2,CONCACAF,22,1563.50,1558.35,-1,CAF,0.000000,2022,12,3,335
26626,2022-12-05,FIFA World Cup,True,0,1,1841.30,1837.56,0,CONMEBOL,28,1530.30,1526.02,0,AFC,0.000000,2022,12,0,339
26587,2022-11-24,FIFA World Cup,True,1,14,1638.71,1640.95,1,CONMEBOL,28,1530.30,1526.02,0,AFC,0.000000,2022,11,3,328
26607,2022-11-29,FIFA World Cup,True,2,19,1569.82,1582.13,0,UEFA,5,1728.47,1737.46,0,UEFA,0.000000,2022,11,1,333


In [111]:
#now I can delete data feature, everything was extracted :)
matches_merged.drop("date", axis = 1, inplace = True)

In [112]:
#time to label encoding
#quick look on tournament feature.Well...that's many categories. To many to just One Hot Encoding
#maybe good idea is to reduce number of different categories
matches_merged.tournament.value_counts()

Friendly                                8557
FIFA World Cup qualification            5531
UEFA Euro qualification                 1724
African Cup of Nations qualification    1272
AFC Asian Cup qualification              496
                                        ... 
CONMEBOL–UEFA Cup of Champions             2
Copa Confraternidad                        1
ELF Cup                                    1
TIFOCO Tournament                          1
Three Nations Cup                          1
Name: tournament, Length: 84, dtype: int64

In [113]:
#let's One hot encode all tournaments where number of matches > 100 and all others put in one "other" category
counts = matches_merged.tournament.value_counts()
def tournament_groupby(tourn, value):
    if counts[tourn] <= value:
        return "other"
    else:
        return tourn

In [114]:
matches_merged["tournament"] = matches_merged["tournament"].apply(lambda row: tournament_groupby(row, 100))

In [115]:
#much better :)
matches_merged.tournament.value_counts()

Friendly                                8557
FIFA World Cup qualification            5531
other                                   1752
UEFA Euro qualification                 1724
African Cup of Nations qualification    1272
AFC Asian Cup qualification              496
African Cup of Nations                   491
FIFA World Cup                           488
UEFA Nations League                      468
Copa América                             320
CECAFA Cup                               308
Gold Cup                                 299
COSAFA Cup                               293
AFF Championship                         258
UEFA Euro                                253
AFC Asian Cup                            227
African Nations Championship             223
Gulf Cup                                 204
CFU Caribbean Cup qualification          177
UNCAF Cup                                151
SAFF Cup                                 144
Confederations Cup                       136
Name: tour

In [116]:
#and let's see confederation_1 column
matches_merged.confederation_1.value_counts()
#not bad :)

UEFA        7768
CAF         5748
AFC         5446
CONCACAF    2786
CONMEBOL    1647
OFC          377
Name: confederation_1, dtype: int64

In [117]:
#time to encoding with pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

matches_merged['neutral'].replace([False, True], [0, 1], inplace = True)

encoder_pipe = ColumnTransformer(transformers=[
    ("enc", OneHotEncoder(), ["tournament", "confederation_1", "confederation_2"])], remainder = 'passthrough')

In [118]:
encoder_pipe.fit_transform(matches_merged)[0]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        3.90000000e+01,  3.40000000e+01,  0.00000000e+00,  0.00000000e+00,
        6.90000000e+01,  2.20000000e+01,  0.00000000e+00,  0.00000000e+00,
       -3.59005318e+02,  1.99300000e+03,  1.00000000e+00,  4.00000000e+00,
        1.00000000e+00])

## Building model

In [119]:
#checking distribution of target variable
matches_merged["result"].value_counts(normalize=True)


0    0.480692
2    0.277049
1    0.242260
Name: result, dtype: float64

In [120]:
#well, our data is unbalanced
#it is to much wins for team1. Let's balance wins

columns1 = ['rank_1', 'total_points_1','previous_points_1', 'rank_change_1', 'confederation_1']
columns2 = ['rank_2','total_points_2', 'previous_points_2', 'rank_change_2','confederation_2']
threshold = (0.480607 - 0.276745)/(2 * 0.480607)

random_indexes = matches_merged[matches_merged.result==0].sample(frac=threshold ,random_state=42).index
matches_merged.loc[random_indexes, columns1 + columns2] = matches_merged.loc[random_indexes, columns2 + columns1].values
matches_merged.loc[random_indexes, "result"] = 2


In [121]:
#now is better :)
matches_merged["result"].value_counts(normalize=True)

2    0.379017
0    0.378723
1    0.242260
Name: result, dtype: float64

In [122]:
#we are interested in the win of a given team, so in order to further balance the data, ties can be removed
matches_merged = matches_merged[matches_merged.result != 1]
matches_merged["result"].replace([0, 2], [0, 1], inplace = True)

In [124]:
# matches_merged.to_csv(r".\data\world cup 2022\final_set.csv", index=False)
final_set = pd.read_csv(r".\data\world cup 2022\final_set.csv")

In [125]:
#time to divide df to training set and target variable
y = final_set["result"].copy()
X = final_set.drop("result", axis=1)

In [126]:
#spliting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [127]:
#making feature weights (matches 10 year ago are for sure much less important than newer ones)
weights_train = X_train.time_delta
weights_test = X_test.time_delta

weights_train = (weights_train.min()*-1) - weights_train
weights_test = (weights_test.min()*-1) - weights_test

In [128]:
#importing algorithms
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

gb_clf = GradientBoostingClassifier()
rf_clf = RandomForestClassifier()
xgb_clf = XGBClassifier()

C:\Users\Laptop\anaconda3\envs\portfolio\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [129]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

model_pipeline = Pipeline([
    ("encoder", encoder_pipe),
    ("scaler", MinMaxScaler()),
    ("classifier", rf_clf)
])


params_xgb=[
    {
        "scaler": [None],
        "classifier": [xgb_clf],
        "classifier__max_depth": [4,5, 6],
        "classifier__learning_rate": [0.1],
    }
]

params_rfc = [
        {
        "scaler": [MinMaxScaler(), StandardScaler()],
        "classifier": [rf_clf],
        "classifier__max_depth": [9,10,11]
    }
]

params_gb=[
    {
        "scaler": [StandardScaler()],
        "classifier": [gb_clf],
        'classifier__max_depth' : [3,4,5],
#       "classifier__learning_rate": [0.1],
    }
]


In [130]:
model_xgb = GridSearchCV(model_pipeline, params_xgb, cv=3, scoring = "accuracy", n_jobs=5)

In [131]:
model_rfc = GridSearchCV(model_pipeline, params_rfc, cv=3, scoring = "accuracy", n_jobs=5)

In [132]:
model_gb = GridSearchCV(model_pipeline, params_gb, cv=3, scoring = "accuracy", n_jobs=5)

In [133]:
model_xgb.fit(X_train, y_train, classifier__sample_weight=weights_train)


[22:24:16] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('encoder',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('enc',
                                                                         OneHotEncoder(),
                                                                         ['tournament',
                                                                          'confederation_1',
                                                                          'confederation_2'])])),
                                       ('scaler', MinMaxScaler()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=5,
             param_grid=[{'classifier': [XGBClassifier(base_score=None,
                                                       booster=None,
                                                       colsample_byle...
                                                       missing=nan,
                                                       monotone_constraints=None,
                                                       n_estimators=100,
                                                       n_jobs=None,
                                                       num_parallel_tree=None,
                                                       predictor=None,
                                                       random_state=None,
                                                       reg_alpha=None,
                                                       reg_lambda=None,
                                                       scale_pos_weight=None,
                                                       subsample=None,
                                                       tree_method=None,
                                                       validate_parameters=None,
                                                       verbosity=None)],
                          'classifier__learning_rate': [0.1],
                          'classifier__max_depth': [4, 5, 6],
                          'scaler': [None]}],
             scoring='accuracy')

In [134]:
model_rfc.fit(X_train, y_train, classifier__sample_weight=weights_train)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('encoder',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('enc',
                                                                         OneHotEncoder(),
                                                                         ['tournament',
                                                                          'confederation_1',
                                                                          'confederation_2'])])),
                                       ('scaler', MinMaxScaler()),
                                       ('classifier',
                                        RandomForestClassifier(max_depth=11))]),
             n_jobs=5,
             param_grid=[{'classifier': [RandomForestClassifier(max_depth=11)],
                          'classifier__max_depth': [9, 10, 11],
                          'scaler': [MinMaxScaler(), StandardScaler()]}],
             scoring='accuracy')

In [135]:
model_gb.fit(X_train, y_train, classifier__sample_weight=weights_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('encoder',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('enc',
                                                                         OneHotEncoder(),
                                                                         ['tournament',
                                                                          'confederation_1',
                                                                          'confederation_2'])])),
                                       ('scaler', MinMaxScaler()),
                                       ('classifier',
                                        RandomForestClassifier(max_depth=11))]),
             n_jobs=5,
             param_grid=[{'classifier': [GradientBoostingClassifier()],
                          'classifier__max_depth': [3, 4, 5],
                          'scaler': [StandardScaler()]}],
             scoring='accuracy')

In [136]:
#XGB
grid_search_table = pd.DataFrame(data=model_xgb.cv_results_)
grid_search_table[grid_search_table["rank_test_score"] < 5].sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__learning_rate,param_classifier__max_depth,param_scaler,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.811332,0.011443,0.028334,0.004497,"XGBClassifier(base_score=None, booster=None, c...",0.1,4,None,"{'classifier': XGBClassifier(base_score=None, ...",0.720200,0.729955,0.718059,0.722738,0.005178,1
1,2.509666,0.101378,0.029333,0.001885,"XGBClassifier(base_score=None, booster=None, c...",0.1,5,None,"{'classifier': XGBClassifier(base_score=None, ...",0.716631,0.727100,0.718772,0.720834,0.004516,2
2,2.873999,0.219071,0.027005,0.000823,"XGBClassifier(base_score=None, booster=None, c...",0.1,6,None,"{'classifier': XGBClassifier(base_score=None, ...",0.714966,0.731620,0.709255,0.718614,0.009488,3


In [137]:
#RFC
grid_search_table = pd.DataFrame(data=model_rfc.cv_results_)
grid_search_table[grid_search_table["rank_test_score"] < 5].sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__max_depth,param_scaler,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
4,2.338667,0.020285,0.183667,0.000474,RandomForestClassifier(max_depth=11),11,MinMaxScaler(),{'classifier': RandomForestClassifier(max_dept...,0.720914,0.734237,0.711159,0.722103,0.009459,1
5,2.022321,0.046909,0.130333,0.006592,RandomForestClassifier(max_depth=11),11,StandardScaler(),{'classifier': RandomForestClassifier(max_dept...,0.718534,0.730431,0.714966,0.721310,0.006612,2
2,1.891334,0.056308,0.168665,0.001701,RandomForestClassifier(max_depth=11),10,MinMaxScaler(),{'classifier': RandomForestClassifier(max_dept...,0.713062,0.734237,0.715203,0.720834,0.009518,3
1,1.942999,0.102531,0.162006,0.004317,RandomForestClassifier(max_depth=11),9,StandardScaler(),{'classifier': RandomForestClassifier(max_dept...,0.713300,0.734475,0.714490,0.720755,0.009714,4


In [138]:
#GBC
grid_search_table = pd.DataFrame(data=model_gb.cv_results_)
grid_search_table[grid_search_table["rank_test_score"] < 5].sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__max_depth,param_scaler,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,5.477011,0.012581,0.039322,0.006614,GradientBoostingClassifier(),3,StandardScaler(),"{'classifier': GradientBoostingClassifier(), '...",0.715679,0.732096,0.723055,0.723610,0.006714,1
1,6.483998,0.599658,0.038665,0.004188,GradientBoostingClassifier(),4,StandardScaler(),"{'classifier': GradientBoostingClassifier(), '...",0.716155,0.728051,0.724483,0.722896,0.004984,2
2,6.793011,0.261547,0.028328,0.006234,GradientBoostingClassifier(),5,StandardScaler(),"{'classifier': GradientBoostingClassifier(), '...",0.711159,0.730193,0.711635,0.717662,0.008863,3


In [139]:
print(f"best train accuracy for XGBoost model: {model_xgb.best_score_: .3f}%")
print(f"best train accuracy for Random Forest Classifier: {model_rfc.best_score_: .3f}%")
print(f"best train accuracy for Gradient Boosting Classifier: {model_gb.best_score_: .3f}%")    

best train accuracy for XGBoost model:  0.723%
best train accuracy for Random Forest Classifier:  0.722%
best train accuracy for Gradient Boosting Classifier:  0.724%


In [140]:
from sklearn.base import clone
from sklearn.metrics import accuracy_score
#XGBoost
test_xgb = clone(model_xgb.best_estimator_)
test_xgb.fit(X_train, y_train, classifier__sample_weight=weights_train)
y_pred_xgb = test_xgb.predict(X_test)


[22:25:47] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [141]:
#Random Forest
test_rfc = clone(model_rfc.best_estimator_)
test_rfc.fit(X_train, y_train)
y_pred_rfc = test_rfc.predict(X_test)

In [142]:
#GBC
test_gb = clone(model_gb.best_estimator_)
test_gb.fit(X_train, y_train, classifier__sample_weight=weights_train)
y_pred_gb = test_gb.predict(X_test)

In [143]:
print(f"Test set accuracy for XGBoost model: {accuracy_score(y_test, y_pred_xgb): .4f}%")
print(f"Test set accuracy for Random Forest Classifier: {accuracy_score(y_test, y_pred_rfc): .4f}%")
print(f"Test set accuracy for Gradient Boosting Classifier: {accuracy_score(y_test, y_pred_gb): .4f}%")

Test set accuracy for XGBoost model:  0.7169%
Test set accuracy for Random Forest Classifier:  0.7222%
Test set accuracy for Gradient Boosting Classifier:  0.7215%


## Final models


In [144]:
#instead of making one final model I want to use all models trained on whole data set
weights_all = X.time_delta
weights_all = (weights_all.min()*-1) - weights_all

final_xgb_model= clone(model_xgb.best_estimator_)
final_rfc_model= clone(model_rfc.best_estimator_)
final_gbc_model= clone(model_gb.best_estimator_)


final_xgb_model.fit(X, y, classifier__sample_weight=weights_all)
final_rfc_model.fit(X, y, classifier__sample_weight=weights_all)
final_gbc_model.fit(X, y, classifier__sample_weight=weights_all)

[22:26:00] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc', OneHotEncoder(),
                                                  ['tournament',
                                                   'confederation_1',
                                                   'confederation_2'])])),
                ('scaler', StandardScaler()),
                ('classifier', GradientBoostingClassifier())])

## Making real-life predictions with all models

In [145]:
#as a match example I will use match Poland-France from world cup 2022
#Poland is team_1 and France team_2

match_example = ["FIFA World Cup", 1,26, 1548.59, 1546.18, 0, "UEFA", 4, 1759.78, 1764.85,0, "UEFA",0,2022,12,6,338]
match_example = pd.DataFrame(match_example, index = X.columns)
match_example.T

,tournament,neutral,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2,time_delta,year,month,day_of_week,day_of_year
0,FIFA World Cup,1,26,1548.59,1546.18,0,UEFA,4,1759.78,1764.85,0,UEFA,0,2022,12,6,338


In [146]:
result_xgb = final_xgb_model.predict_proba(match_example.transpose())
result_rfc = final_rfc_model.predict_proba(match_example.transpose())
result_gbc = final_gbc_model.predict_proba(match_example.transpose())

In [147]:
print("XGB model predictions:")
print(f"Win chance for Poland: {result_xgb[0,0] * 100:.2f}%")
print(f"Win chance for France: {result_xgb[0,1] * 100:.2f}%")
print()
print("RFC model predictions:")
print(f"Win chance for Poland: {result_rfc[0,0] * 100:.2f}%")
print(f"Win chance for France: {result_rfc[0,1] * 100:.2f}%")
print()
print("GBC model predictions:")
print(f"Win chance for Poland: {result_gbc[0,0] * 100:.2f}%")
print(f"Win chance for France: {result_gbc[0,1] * 100:.2f}%")

XGB model predictions:
Win chance for Poland: 31.06%
Win chance for France: 68.94%

RFC model predictions:
Win chance for Poland: 41.81%
Win chance for France: 58.19%

GBC model predictions:
Win chance for Poland: 22.35%
Win chance for France: 77.65%


In [148]:
#Poland-Mexico
match_example2 = ["FIFA World Cup", 1,26, 1548.59, 1546.18, 0, "UEFA", 13, 1644.89, 1649.57 , -1, "CONCACAF",0,2022,12,6,338]
match_example2 = pd.DataFrame(match_example2, index = X.columns)
match_example2.T

,tournament,neutral,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2,time_delta,year,month,day_of_week,day_of_year
0,FIFA World Cup,1,26,1548.59,1546.18,0,UEFA,13,1644.89,1649.57,-1,CONCACAF,0,2022,12,6,338


In [149]:
result2_xgb = final_xgb_model.predict_proba(match_example2.transpose())
result2_rfc = final_rfc_model.predict_proba(match_example2.transpose())
result2_gbc = final_gbc_model.predict_proba(match_example2.transpose())

In [150]:
print("XGB model predictions:")
print(f"Win chance for Poland: {result2_xgb[0,0] * 100:.2f}%")
print(f"Win chance for Mexico: {result2_xgb[0,1] * 100:.2f}%")
print()
print("RFC model predictions:")
print(f"Win chance for Poland: {result2_rfc[0,0] * 100:.2f}%")
print(f"Win chance for Mexico: {result2_rfc[0,1] * 100:.2f}%")
print()
print("GBC model predictions:")
print(f"Win chance for Poland: {result2_gbc[0,0] * 100:.2f}%")
print(f"Win chance for Mexico: {result2_gbc[0,1] * 100:.2f}%")

XGB model predictions:
Win chance for Poland: 18.30%
Win chance for Mexico: 81.70%

RFC model predictions:
Win chance for Poland: 46.84%
Win chance for Mexico: 53.16%

GBC model predictions:
Win chance for Poland: 19.95%
Win chance for Mexico: 80.05%


## Building one model using voting classifier

In [151]:
#All model have different win-chance probabilities
#maybe merge all predictions together would be much better
from sklearn.ensemble import VotingClassifier

all_models_classifier = VotingClassifier(estimators=[
    ('xgb', final_xgb_model), ('rfc', final_rfc_model), ('gbc', final_gbc_model)], voting='soft')

all_models_classifier.fit(X, y)

[22:26:43] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('xgb',
                              Pipeline(steps=[('encoder',
                                               ColumnTransformer(remainder='passthrough',
                                                                 transformers=[('enc',
                                                                                OneHotEncoder(),
                                                                                ['tournament',
                                                                                 'confederation_1',
                                                                                 'confederation_2'])])),
                                              ('scaler', None),
                                              ('classifier',
                                               XGBClassifier(base_score=0.5,
                                                             booster='gbtree',
                                                             colsample_bylevel=1,
                                                             colsample_bynode=1,
                                                             colsample_bytree=1,
                                                             enable_categorical=...
                                                                                 'confederation_2'])])),
                                              ('scaler', MinMaxScaler()),
                                              ('classifier',
                                               RandomForestClassifier(max_depth=11))])),
                             ('gbc',
                              Pipeline(steps=[('encoder',
                                               ColumnTransformer(remainder='passthrough',
                                                                 transformers=[('enc',
                                                                                OneHotEncoder(),
                                                                                ['tournament',
                                                                                 'confederation_1',
                                                                                 'confederation_2'])])),
                                              ('scaler', StandardScaler()),
                                              ('classifier',
                                               GradientBoostingClassifier())]))],
                 voting='soft')

In [174]:
#saving model
import pickle
pickle.dump(all_models_classifier, open(r".\data\world cup 2022\all_models_classifier.sav", 'wb'))

In [152]:
voting_results = all_models_classifier.predict_proba(match_example.transpose())
voting_results2 = all_models_classifier.predict_proba(match_example2.transpose())

print("Voting Classifier model predictions:")
print()
print("Poland-France")
print(f"Win chance for Poland: {voting_results[0,0] * 100:.2f}%")
print(f"Win chance for France: {voting_results[0,1] * 100:.2f}%")
print()
print("Poland-Mexico")
print(f"Win chance for Poland: {voting_results2[0,0] * 100:.2f}%")
print(f"Win chance for Mexico: {voting_results2[0,1] * 100:.2f}%")


Voting Classifier model predictions:

Poland-France
Win chance for Poland: 34.34%
Win chance for France: 65.66%

Poland-Mexico
Win chance for Poland: 38.94%
Win chance for Mexico: 61.06%


## Other matches:

### Morocco-Portugal

In [153]:
morocco_portugal_match = ["FIFA World Cup", 1,22, 1563.5, 1558.35, 1, "CAF", 9, 1676.56, 1678.65 , 0, "UEFA",0,2022,12,5,343]
morocco_portugal_match = pd.DataFrame(morocco_portugal_match, index = X.columns)
morocco_portugal_match.T

,tournament,neutral,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2,time_delta,year,month,day_of_week,day_of_year
0,FIFA World Cup,1,22,1563.5,1558.35,1,CAF,9,1676.56,1678.65,0,UEFA,0,2022,12,5,343


In [154]:
morocco_portugal_voting_results = all_models_classifier.predict_proba(morocco_portugal_match.transpose())

print("Voting Classifier model predictions:")
print()
print("Morocco - Portugal")
print(f"Win chance for Morocco: {morocco_portugal_voting_results[0,0] * 100:.2f}%")
print(f"Win chance for Portugal: {morocco_portugal_voting_results[0,1] * 100:.2f}%")

Voting Classifier model predictions:

Morocco - Portugal
Win chance for Morocco: 36.72%
Win chance for Portugal: 63.28%


## Mini app for entering matches faster


In [ ]:
first_date = fifa_ranking.sort_values("date").iloc[0].date
matches = matches[matches.date >= first_date]
first_date

In [3]:
import pickle
columns = ['tournament', 'neutral', 'rank_1', 'total_points_1',
       'previous_points_1', 'rank_change_1', 'confederation_1', 'rank_2',
       'total_points_2', 'previous_points_2', 'rank_change_2',
       'confederation_2', 'time_delta', 'year', 'month', 'day_of_week',
       'day_of_year']


def predict_match_result(first_team, second_team, date, tournament, if_neutral=True):
    fifa_ranking = pd.read_csv(r"data\world cup 2022\fifa_ranking-2022-10-06.csv")
    fifa_ranking["rank_date"] = pd.to_datetime(fifa_ranking["rank_date"], format = '%Y-%m-%d')
    first_team_data = fifa_ranking[fifa_ranking.country_full == first_team].sort_values("rank_date", ascending=False).iloc[0]
    second_team_data = fifa_ranking[fifa_ranking.country_full == second_team].sort_values("rank_date", ascending=False).iloc[0]
    
    date = pd.to_datetime(date, format = '%Y-%m-%d')
    
    match = [tournament, if_neutral,
             first_team_data["rank"], first_team_data["total_points"], first_team_data["previous_points"], first_team_data["rank_change"], first_team_data["confederation"],
            second_team_data["rank"], second_team_data["total_points"], second_team_data["previous_points"], second_team_data["rank_change"], second_team_data["confederation"],
            0, date.year, date.month, date.dayofweek, date.dayofyear]
    
    match = pd.DataFrame(match, index = columns)
    
    model = pickle.load(open(r".\data\world cup 2022\all_models_classifier.sav", 'rb'))
    result = model.predict_proba(match.transpose())
    
    print("Model predictions:")
    print()
    print(f"{first_team} - {second_team}")
    print(f"Win chance for {first_team}: {result[0,0] * 100:.2f}%")
    print(f"Win chance for {second_team}: {result[0,1] * 100:.2f}%")

### FIFA WORLD CUP 2022 FINAL

In [4]:
predict_match_result("France", "Argentina", "2022-12-18", "FIFA World Cup", if_neutral=True)
#real result: France 3:3 Argentina  win for Argentina in penalties     good prediction :)

C:\Users\Laptop\anaconda3\envs\portfolio\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Model predictions:

France - Argentina
Win chance for France: 44.63%
Win chance for Argentina: 55.37%


### FIFA WORLD CUP 2022 Third-place play-off

In [213]:
predict_match_result("Croatia", "Morocco", "2022-12-17", "FIFA World Cup", if_neutral=True)
#real result: Croatia 2:1 Morocco     good prediction :)

Model predictions:

Croatia - Morocco
Win chance for Croatia: 58.78%
Win chance for Morocco: 41.22%
